In [2]:
# %pip install --upgrade pip
# %pip install fairseq2 sonar-space
# %pip install libsndfile
%pip install transformers datasets evaluate accelerate trl
%pip install mteb sentence-transformers


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import Libraries and Utilities
import torch

# Import Encoder Modules
from Encoders.text.BART import BART as BART_ENC
from Encoders.text.GPT import GPT_2 as GPT_ENC
from Encoders.text.SEED import SEED

from Training.encoder_trainers import Contrast

from Training.Data_Wrangling.text import load_local

from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

In [12]:
# try:
#     sts_dataset = load_local('SEED_ENCODED_MTEB')
#     print(f"\nSuccessfully loaded 'mteb/stsbenchmark-sts' (English) Test set. Number of examples: {len(sts_dataset)}")
# except Exception as e:
#     print(f"\nCould not load mteb/stsbenchmark-sts dataset: {e}")

# checkpoint = 'facebook/bart-base'
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sts_dataset = load_dataset("mteb/stsbenchmark-sts", split="train") 

In [13]:
print(sts_dataset)
print(sts_dataset[0])

Dataset({
    features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
    num_rows: 5749
})
{'split': 'train', 'genre': 'main-captions', 'dataset': 'MSRvid', 'year': '2012test', 'sid': '0001', 'score': 5.0, 'sentence1': 'A plane is taking off.', 'sentence2': 'An air plane is taking off.'}


In [25]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_local("Training/Data_Wrangling/SEED_ENC_MTEB_DATASET")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `list[str]` (batch or single pretokenized example) or `list[list[str]]` (batch of pretokenized examples).

In [ ]:
# https://docs.pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html